# 데이터 분할

데이터를 훈련 세트와 테스트 세트로 분할한다.    
데이터를 분할하는 것은 모델의 성능을 검증하는데 매우 중요하므로 데이터 누수와 같은 문제가 발생하지 않도록 각별한 주의가 필요하다.

In [7]:
import pandas as pd
import spacy
import umap
import numpy as np 
from pathlib import Path
import sys
sys.path.append("..")
import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

from ml_editor.data_processing import format_raw_df, get_random_train_test_split, get_vectorized_inputs_and_label, get_split_by_author

data_path = Path('./data/processed/writers/writers.csv')
df = pd.read_csv(data_path, index_col=0)
df = format_raw_df(df.copy())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 랜덤 분할

가장 간단한 방법: 기준없이 랜덤하게 나눈다.

In [9]:
train_df_rand, test_df_rand = get_random_train_test_split(df[df["is_question"]], test_size=0.3, random_state=40)

In [10]:
print("훈련 세트: %s개 질문, 테스트 세트: %s개 질문" % (len(train_df_rand),len(test_df_rand)))
train_owners = set(train_df_rand['OwnerUserId'].values)
test_owners = set(test_df_rand['OwnerUserId'].values)
print("훈련 세트에 있는 작성자: %s명" % len(train_df_rand))
print("테스트 세트에 있는 작성자: %s명" % len(test_df_rand))
print("양쪽에 모두 등장하는 작성자: %s명" % len(train_owners.intersection(test_owners)))

훈련 세트: 7911개 질문, 테스트 세트: 3391개 질문
훈련 세트에 있는 작성자: 7911명
테스트 세트에 있는 작성자: 3391명
양쪽에 모두 등장하는 작성자: 758명


데이터를 분할하기 매우 쉬운 방법이지만, 같은 작성자가 쓴 질문들의 경우 유사한 패턴이 있을 수 있다.    
따라서 같은 작성자가 작성한 질문들을 훈련과 테스트 세트에 모두 분배할 경우 작성자를 특정하여 예측을 수행하는 데이터 누수 문제가 발생할 수 있다.
=> 데이터 분할 시 질문 단위가 아니라 작성자 단위로 분할하는 것이 좋다.

# 작성자 기준으로 분할

In [11]:
train_author, test_author = get_split_by_author(df[df["is_question"]], test_size=0.3, random_state=40)

print("훈련 세트: %s개 질문, 테스트 세트: %s개 질문" % (len(train_author),len(test_author)))
train_owners = set(train_author['OwnerUserId'].values)
test_owners = set(test_author['OwnerUserId'].values)
print("훈련 세트에 있는 작성자: %s명" % len(train_owners))
print("훈련 세트에 있는 작성자: %s명" % len(test_owners))
print("양쪽에 모두 등장하는 작성자: %s명" % len(train_owners.intersection(test_owners)))

훈련 세트: 8161개 질문, 테스트 세트: 3141개 질문
훈련 세트에 있는 작성자: 3752명
훈련 세트에 있는 작성자: 1608명
양쪽에 모두 등장하는 작성자: 0명
